# Sample client for the MDT research environment

### 1. Importing dependencies and setting up the vantage6 client

In [1]:
from vantage6.client import UserClient as Client

# Note: we assume here the config.py you just created is in the current directory.
# If it is not, then you need to make sure it can be found on your PYTHONPATH
import config
import json
from config_reader import read_config
from output_encoders import decode_files


# Initialize the client object, and run the authentication
client = Client(config.server_url, config.server_port, config.server_api,
                log_level='debug')
client.authenticate(config.username, config.password)

# Optional: setup the encryption, if you have an organization_key
client.setup_encryption(config.organization_key)

 Welcome to
                  _                     __  
                 | |                   / /  
__   ____ _ _ __ | |_ __ _  __ _  ___ / /_  
\ \ / / _` | '_ \| __/ _` |/ _` |/ _ \ '_ \ 
 \ V / (_| | | | | || (_| | (_| |  __/ (_) |
  \_/ \__,_|_| |_|\__\__,_|\__, |\___|\___/ 
                            __/ |           
                           |___/            

 --> Join us on Discord! https://discord.gg/rwRvwyK
 --> Docs: https://docs.vantage6.ai
 --> Blog: https://vantage6.ai
------------------------------------------------------------
Cite us!
If you publish your findings obtained using vantage6, 
please cite the proper sources as mentioned in:
https://vantage6.ai/vantage6/references
------------------------------------------------------------
Authenticating user mdt-researcher...
Starting new HTTPS connection (1): v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443
https://v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443 "POST /api/token/user HTTP/1.1" 200 750
Successfull

### 2. Define the algorithm input columns and hyper-parameters

In [2]:
predictor_cols = ['GENDER', 'T2D_STATUS', 'SMOKING_STATUS', 'SMOKING_QUANTITY',  
'TOTAL_CHOLESTEROL_VALUE', 'HDL_CHOLESTEROL_VALUE', 'LDL_CHOLESTEROL_VALUE', 
'SYSTOLIC_VALUE', 'DIASTOLIC_VALUE', 'PLASMA_ALBUNIM_VALUE', 'EGFR_VALUE', 
'HEMOGLOBIN_VALUE', 'HYPERTENSION_STATUS', 'CREATININE_VALUE', 'HBA1C_VALUE', 'AGE']

outcome_cols = ['LENFOL', 'FSTAT']

num_update_iter = 21

dl_config = read_config("lifelines_ci.ini")

n_fold = 10
fold_index = 4
output_pth = "aggregated_weights.pth"

### 3. Create and send an algorithm execution request

In [3]:
model_training_task = client.task.create(
   collaboration=2, #mdt-consortium/aggregator node@SURF
   # Must be set to the 'aggregator' organization
   organizations=[7], #mdt-consortium/aggregator node@SURF
   name="federated_model_training_poc",   

   image="ghcr.io/mydigitwinnl/federated_cvdm_training_poc:82b6390525e5690a56c1ddde3dd2a2eb2d98b1c8",
   description='',
   input_= {
      "method":"central_ci",
      "kwargs": {
            "predictor_cols": predictor_cols,
            "outcome_cols": outcome_cols,
            "dl_config": dl_config,
            "num_update_iter": num_update_iter,
            "n_fold": n_fold,
            "fold_index": fold_index,
            "agg_weight_filename": output_pth
      }
   },
   databases=[
         {'label': 'lifelines_dummy'} #as configured @ lifelines

   ]
)

Making request: GET | https://v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443/api/organization/7 | None
Starting new HTTPS connection (1): v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443
https://v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443 "GET /api/organization/7 HTTP/1.1" 200 1463
Making request: POST | https://v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443/api/task | None
Starting new HTTPS connection (1): v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443
https://v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443 "POST /api/task HTTP/1.1" 201 761


### 4. Wait for the results

In [4]:
task_id = model_training_task['id']
print('Waiting for results...')

alg_output = client.wait_for_results(task_id)
print('Results received!')

alg_result = json.loads(alg_output['data'][0]['result'])

#parse the result as a json object
alg_result_dict = json.loads(alg_result)

#save the files on /tmp (update this output path as needed)
saved_files = decode_files(alg_result_dict["encoded_output_files"],'/tmp')

print("Decoding and saving output files:")
print(saved_files)

print(f"Iterations: {alg_result_dict['iterations']}")
print(f"Runtime: {alg_result_dict['runtime']} sec.")
print(f"Predictor cols: {alg_result_dict['predictor_cols']}")
print(f"Outcome cols: {alg_result_dict['outcome_cols']}")
print(f"Data nodes: {alg_result_dict['data_nodes']}")
print(f"Aggregator node: {alg_result_dict['aggregator']}")

Waiting for results...
Making request: GET | https://v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443/api/result | {'task_id': 337}
Starting new HTTPS connection (1): v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443
https://v6mdtserver.mydigitwin-umcu.src.surf-hosted.nl:443 "GET /api/result?task_id=337 HTTP/1.1" 200 426130
Results received!
Decoding and saving output files:
['/tmp/aggregated_weights.pth', '/tmp/global_eval_C-statistic_lifelines.png', '/tmp/global_ci_4.npy', '/tmp/local_ci_4.npy']
Iterations: 20
Runtime: 348.3144805431366 sec.
Predictor cols: ['GENDER', 'T2D_STATUS', 'SMOKING_STATUS', 'SMOKING_QUANTITY', 'TOTAL_CHOLESTEROL_VALUE', 'HDL_CHOLESTEROL_VALUE', 'LDL_CHOLESTEROL_VALUE', 'SYSTOLIC_VALUE', 'DIASTOLIC_VALUE', 'PLASMA_ALBUNIM_VALUE', 'EGFR_VALUE', 'HEMOGLOBIN_VALUE', 'HYPERTENSION_STATUS', 'CREATININE_VALUE', 'HBA1C_VALUE', 'AGE']
Outcome cols: ['LENFOL', 'FSTAT']
Data nodes: [3, 4]
Aggregator node: 7
